In [2]:
import glob
import os
import pandas as pd
import numpy as np
import weaviate

In [3]:
client = weaviate.Client("http://localhost:8080/")

In [4]:
client.is_ready()

True

In [5]:
def load_parquet(parquet_path):
    return pd.read_parquet(parquet_path)

In [6]:
def print_parquet_data(data):
    for _, row in data.iterrows():
        print("Abstract:", row['abstract'])
        print("Embedding:", row['embeddings'])
        print("DOI:", row['doi'])
        print()

In [7]:
abstracts1 = load_parquet('arxiv-paper-abstracts/arxiv_abstracts/arxiv_abstracts/arxiv_abstracts/abstracts_1.parquet')

In [12]:
abstracts1.head()

,abstract,embeddings,doi
0,A fully differential calculation in perturba...,"[-0.035151865, 0.022851437, 0.025942933, -0.02...",0704.0001
1,"We describe a new algorithm, the $(k,\ell)$-...","[0.035485767, -0.0015772493, -0.0016615744, -0...",0704.0002
2,The evolution of Earth-Moon system is descri...,"[-0.014510429, 0.010210799, 0.049661566, -0.01...",0704.0003
3,We show that a determinant of Stirling cycle...,"[0.029191103, 0.047992915, -0.0061754594, -0.0...",0704.0004
4,In this paper we show how to compute the $\L...,"[-0.015174898, 0.01603887, 0.04062805, -0.0246...",0704.0005
